### Setup

First, import the required packages.

Next, specify some important variables:
* `platform_address`: The address of the Yugabyte Platform API
* `platform_api_key`: The API key used to authenticate with the Platform API

Finally, open a HTTP connection to the Yugabyte Platform API.

In [55]:
import os
import http.client
import json
from pprint import pprint

platform_address = os.getenv('API_BASE_URL', "portal.dev.yugabyte.com")
platform_api_key = os.getenv('YB_API_KEY')

conn = http.client.HTTPConnection(f"{platform_address}")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{platform_api_key}"
}

### Get Session Info

Make an API call to session endpoint to determine customer UUID.

In [56]:
conn.request("GET", "/api/v1/session_info", headers=headers)

res = conn.getresponse()
data = res.read()
session_info = json.loads(data)
customer_uuid = session_info['customerUUID']

print('Customer UUID:\n%s' % customer_uuid)

Customer UUID:
11d78d93-1381-4d1d-8393-ba76f47ba7a6


### Get Provider UUID By Name

Make an API call to the provider endpoint to determine provider UUID and regions.

In [43]:
provider_name = 'my-aws-provider' # my-gcp-rovider 
url = f"/api/v1/customers/{customer_uuid}/providers"
conn.request("GET", url, headers=headers)
res = conn.getresponse()

provider_list = json.load(res)
for provider in provider_list:
    if provider['name'] == provider_name and provider['active'] == True:
        provider_uuid = provider['uuid']
        region_list = [region['uuid'] for region in provider['regions']]
        break

print('Provider UUID:\n%s' % provider_uuid)
print('Regions:\n[%s]' % ', '.join(region_list))

Provider UUID:
3ef96e2b-4940-407a-ae55-e13971285791
Regions:
[3b93d3c3-dae2-4f92-95b6-603ea0662d5f]


### Get Access Key

Make an API call to the access key endpoint to determine access key for provider.

In [44]:
url = f"/api/v1/customers/{customer_uuid}/providers/{provider_uuid}/access_keys"
conn.request("GET", url, headers=headers)
res = conn.getresponse()

access_key_list = json.load(res)
access_key_code = access_key_list[0]['idKey']['keyCode']
print('Access Key:\n%s' % access_key_code)

Access Key:
yb-15-my-aws-provider_3ef96e2b-4940-407a-ae55-e13971285791-key


### Get Instance Types Available

Make an API call to see the instance types available for the provider.

In [60]:
url = f"/api/v1/customers/{customer_uuid}/providers/{provider_uuid}/instance_types"
conn.request("GET", url, headers=headers)
res = conn.getresponse()

instance_type = None
for i in json.load(res):
    print(i['instanceTypeCode'])
    if i['instanceTypeCode'] in ['c5.large', 'n1-standard-1']: # desired aws or gcp type code
        print('^^^^^^^ Found !!!!!!!!!!!')
        instance_type = i
        break

pprint(instance_type['instanceTypeDetails'])

m6g.xlarge
m6g.12xlarge
c5d.18xlarge
c5.12xlarge
m6g.large
c4.large
c5d.2xlarge
c5.4xlarge
i3.8xlarge
i3.4xlarge
c5.24xlarge
c4.4xlarge
c4.xlarge
c4.2xlarge
m6g.2xlarge
c5d.large
c5.2xlarge
c5d.12xlarge
i3.large
m6g.4xlarge
c5.large
^^^^^^^ Found !!!!!!!!!!!
{'tenancy': 'Shared',
 'volumeDetailsList': [{'mountPath': '/mnt/d0',
                        'volumeSizeGB': 250,
                        'volumeType': 'EBS'}]}


### Define Universe

In this example, we will create a GCP universe. Define a `Universe` object with the desired configuration.

In [59]:
new_universe = {
    'clusters': [
        {
            'clusterType': 'PRIMARY',
            'userIntent': {
                'universeName': 'my-aws-universe', # or my-gcp-universe
                'providerType': 'aws',
                'provider': provider_uuid,
                'regionList': region_list,
                'numNodes': 3,
                'replicationFactor': 3,
                'instanceType': instance_type['instanceTypeCode'],
                'deviceInfo': {
                    'numVolumes': 1,
                    'volumeSize': instance_type['instanceTypeDetails']['volumeDetailsList'][0]['volumeSizeGB'],
                    'storageType': 'GP2', # 'storageType': 'Persistent',
                },
                'assignPublicIP': True,
                'useTimeSync': True,
                'enableYSQL': True,
                'enableYEDIS': False,
                'enableNodeToNodeEncrypt': True,
                'enableClientToNodeEncrypt': True,
                'enableVolumeEncryption': False,
                'ybSoftwareVersion': '2.7.3.0-b80',
                'accessKeyCode': access_key_code,
                'tserverGFlags': {},
                'masterGFlags': {},
            }
        },
    ],
}

### Create Universe

Make API call to create new universe.

In [58]:
url = f"/api/v1/customers/{customer_uuid}/universes/clusters"
conn.request("POST", url, json.dumps(new_universe), headers)
res = conn.getresponse()
pprint(json.load(res))

{'resourceUUID': '8343ceac-a465-41fa-b509-dd584c312ad6',
 'taskUUID': '440d36f0-8579-4d40-91d6-b81bdbe3a087'}
